In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from PIL import Image, ImageOps
import os
from utils.H264 import *
from utils.ycbcr_conv import *
from utils.check import *
from pathlib import Path
from contextlib import redirect_stdout
import shutil

# delete directory output if exist
if os.path.exists("output"):
    shutil.rmtree("output")

# create directrot output if not exist
if not os.path.exists("output"):
    os.makedirs("output")
if not os.path.exists("output/logs"):
    os.makedirs("output/logs")
if not os.path.exists("output/frames"):
    os.makedirs("output/frames")

if not os.path.exists("output/frames/data_in"):
    os.makedirs("output/frames/data_in")
if not os.path.exists("output/frames/data_in/data_in_uint8"):
    os.makedirs("output/frames/data_in/data_in_uint8")
if not os.path.exists("output/frames/data_in/data_in_hexa"):
    os.makedirs("output/frames/data_in/data_in_hexa")

if not os.path.exists("output/frames/ans"):
    os.makedirs("output/frames/ans")
if not os.path.exists("output/frames/ans/ans_uint8"):
    os.makedirs("output/frames/ans/ans_uint8")
if not os.path.exists("output/frames/ans/ans_hexa"):
    os.makedirs("output/frames/ans/ans_hexa")
if not os.path.exists("output/big_pics"):
    os.makedirs("output/big_pics")


def read_image_into_rgb(path):
    img = Image.open(path)
    img = ImageOps.exif_transpose(img)
    img = img.convert("RGB")
    rgb = np.array(img)
    print(rgb.shape, rgb.dtype)              # (H, W, 3) uint8
    return rgb
def convert_deciaml_to_hexa(num):
    # the num is [0,256), so we need 8 bits to represent it
    hex_num = hex(num)[2:]  # remove the '0x' prefix
    if len(hex_num) == 1:
        hex_num = '0' + hex_num  # pad with leading zero if necessary
    return hex_num
def convert_np_uint8_tensor_to_hexa_file(np_tensor):
    hexa_strings = []
    for line in np_tensor:
        str_hold = ""
        for num in line:
            str_hold += convert_deciaml_to_hexa(num)
            str_hold += " "
        hexa_strings.append(str_hold)
        print(str_hold)

In [2]:
# ipynb_root = Path(__file__).parent
input_img_path = Path("img") / "trump.jpg"
output_img_path = Path("output") / "recover_trump.jpg"
rgb_image = read_image_into_rgb(input_img_path)
ycbcr_image = rgb_2_ycbcr(rgb_image=rgb_image)
recover_rgb_image = ycbcr_2_rgb(ycbcr_image=ycbcr_image)
Image.fromarray(recover_rgb_image, mode="RGB").save(output_img_path, quality=90, optimize=True, progressive=True)

(512, 512, 3) uint8


In [3]:
original_image = read_image_into_rgb(input_img_path)
recover_image = read_image_into_rgb(output_img_path)
diff_two_tensor(A=original_image, B=recover_image)

(512, 512, 3) uint8
(512, 512, 3) uint8
convertion diff: 0.5409317016601562


In [4]:
Y_tensor = ycbcr_image[...,0]
print(Y_tensor.shape, Y_tensor.dtype)

(512, 512) uint8


In [5]:
frames, _ = macro_block_partition(Y_tensor=Y_tensor)
# for each frame, I want to dump it into a txt file
for a in range(frames.shape[0]):
    for b in range(frames.shape[1]):
        tmp_frame = frames[a,b]
        uint_file = f"output/frames/data_in/data_in_uint8/frame_uint8_{a}_{b}.txt"
        hexa_file = f"output/frames/data_in/data_in_hexa/frame_hexa_{a}_{b}.txt"
        print("handling frame", a, b, 'and write file', uint_file, hexa_file)
        with open(uint_file, "w") as f:
            with redirect_stdout(f):
                for i in range(tmp_frame.shape[0]):
                    for j in range(tmp_frame.shape[1]):
                        print(f"{int(tmp_frame[i,j]):4d}", end=' ')
                    print()
        with open(hexa_file, "w") as f:
            with redirect_stdout(f):
                convert_np_uint8_tensor_to_hexa_file(tmp_frame)
        check_uint_file_and_hexa_file(uint_file, hexa_file)
# handle big picture
# take each frames/data_in/data_in_uint8/*.txt and make those txt concat vertic
big_input_uint8_path = Path("output") / "big_pics" / "big_data_in_uint8.txt"
big_input_hexa_path = Path("output") / "big_pics" / "big_data_in_hexa.txt"
with open(big_input_uint8_path, "w") as f:
    with redirect_stdout(f):
        for a in range(frames.shape[0]):
            for b in range(frames.shape[1]):
                uint_file = f"output/frames/data_in/data_in_uint8/frame_uint8_{a}_{b}.txt"
                with open(uint_file, "r") as f_uint:
                    lines = f_uint.readlines()
                    for line in lines:
                        print(line, end='')
with open(big_input_hexa_path, "w") as f:
    with redirect_stdout(f):
        for a in range(frames.shape[0]):
            for b in range(frames.shape[1]):
                hexa_file = f"output/frames/data_in/data_in_hexa/frame_hexa_{a}_{b}.txt"
                with open(hexa_file, "r") as f_hexa:
                    lines = f_hexa.readlines()
                    for line in lines:
                        print(line, end='')
check_uint_file_and_hexa_file(big_input_uint8_path, big_input_hexa_path)

(16, 16, 32, 32) (512, 512)
convertion diff: 0.0
handling frame 0 0 and write file output/frames/data_in/data_in_uint8/frame_uint8_0_0.txt output/frames/data_in/data_in_hexa/frame_hexa_0_0.txt
handling frame 0 1 and write file output/frames/data_in/data_in_uint8/frame_uint8_0_1.txt output/frames/data_in/data_in_hexa/frame_hexa_0_1.txt
handling frame 0 2 and write file output/frames/data_in/data_in_uint8/frame_uint8_0_2.txt output/frames/data_in/data_in_hexa/frame_hexa_0_2.txt
handling frame 0 3 and write file output/frames/data_in/data_in_uint8/frame_uint8_0_3.txt output/frames/data_in/data_in_hexa/frame_hexa_0_3.txt
handling frame 0 4 and write file output/frames/data_in/data_in_uint8/frame_uint8_0_4.txt output/frames/data_in/data_in_hexa/frame_hexa_0_4.txt
handling frame 0 5 and write file output/frames/data_in/data_in_uint8/frame_uint8_0_5.txt output/frames/data_in/data_in_hexa/frame_hexa_0_5.txt
handling frame 0 6 and write file output/frames/data_in/data_in_uint8/frame_uint8_0_6.t

In [6]:
for a in range(frames.shape[0]):
    for b in range(frames.shape[1]):
        log_name = f"output/logs/rec_log_{a}_{b}.txt"
        block_ans = None
        with open(log_name, "w") as f:
            with redirect_stdout(f):
                target_frame = frames[a, b]
                h, w = target_frame.shape
                for i in range(h):
                    for j in range(w):
                        print(f"{target_frame[i, j]:3d}", end=' ')
                    print()
                op_mode = [1, 0, 0, 0]
                QP = 13
                block_ans, _ = consume_a_frame(frame=target_frame, op_mode=op_mode, QP=QP)
        # dump block_ans into a file
        if block_ans is None:
            print("block_ans is None, something wrong!")
        else:
            # output the block ans in both uint8 and hexa
            uint_file = f"output/frames/ans/ans_uint8/ans_uint8_{a}_{b}.txt"
            hexa_file = f"output/frames/ans/ans_hexa/ans_hexa_{a}_{b}.txt"
            print("writing rec file", uint_file, hexa_file)
            with open(uint_file, "w") as f:
                with redirect_stdout(f):
                    for i in range(block_ans.shape[0]):
                        for j in range(block_ans.shape[1]):
                            print(f"{int(block_ans[i,j]):4d}", end=' ')
                        print()
            with open(hexa_file, "w") as f:
                with redirect_stdout(f):
                    convert_np_uint8_tensor_to_hexa_file(block_ans)
            check_uint_file_and_hexa_file(uint_file, hexa_file)

writing rec file output/frames/ans/ans_uint8/ans_uint8_0_0.txt output/frames/ans/ans_hexa/ans_hexa_0_0.txt
writing rec file output/frames/ans/ans_uint8/ans_uint8_0_1.txt output/frames/ans/ans_hexa/ans_hexa_0_1.txt
writing rec file output/frames/ans/ans_uint8/ans_uint8_0_2.txt output/frames/ans/ans_hexa/ans_hexa_0_2.txt
writing rec file output/frames/ans/ans_uint8/ans_uint8_0_3.txt output/frames/ans/ans_hexa/ans_hexa_0_3.txt
writing rec file output/frames/ans/ans_uint8/ans_uint8_0_4.txt output/frames/ans/ans_hexa/ans_hexa_0_4.txt
writing rec file output/frames/ans/ans_uint8/ans_uint8_0_5.txt output/frames/ans/ans_hexa/ans_hexa_0_5.txt
writing rec file output/frames/ans/ans_uint8/ans_uint8_0_6.txt output/frames/ans/ans_hexa/ans_hexa_0_6.txt
writing rec file output/frames/ans/ans_uint8/ans_uint8_0_7.txt output/frames/ans/ans_hexa/ans_hexa_0_7.txt
writing rec file output/frames/ans/ans_uint8/ans_uint8_0_8.txt output/frames/ans/ans_hexa/ans_hexa_0_8.txt
writing rec file output/frames/ans/an

In [7]:
# take each frames/ans/ans_uint8/*.txt and make those txt concat verticall(just line by line) into a big picture
big_ans_uint8_path = Path("output") / "big_pics" / "big_ans_uint8.txt"
big_ans_hexa_path = Path("output") / "big_pics" / "big_ans_hexa.txt"
with open(big_ans_uint8_path, "w") as f:
    with redirect_stdout(f):
        for a in range(frames.shape[0]):
            for b in range(frames.shape[1]):
                ans_file = f"output/frames/ans/ans_uint8/ans_uint8_{a}_{b}.txt"
                with open(ans_file, "r") as f_ans:
                    lines = f_ans.readlines()
                    for line in lines:
                        print(line, end='')
with open(big_ans_hexa_path, "w") as f:
    with redirect_stdout(f):
        for a in range(frames.shape[0]):
            for b in range(frames.shape[1]):
                ans_file = f"output/frames/ans/ans_hexa/ans_hexa_{a}_{b}.txt"
                with open(ans_file, "r") as f_ans:
                    lines = f_ans.readlines()
                    for line in lines:
                        print(line, end='')

In [8]:
# condense hexa txt's ' ' for it is going to be sent into verilog simulator
# handle big picture hexa.txt first
hexa_path = Path("output") / "big_pics" / "big_ans_hexa.txt"
max_content = 16 * 32
with open(hexa_path, "r") as f:
    cnt = 0
    lines = f.readlines()
    condensed_lines = []
    for line in lines:
        condensed_line = line.replace(" ", "")
        condensed_lines.append(condensed_line)
    condensed_hexa_path = Path("output") / "big_pics" / "big_ans_hexa_condensed.txt"
    with open(condensed_hexa_path, "w") as f_condensed:
        with redirect_stdout(f_condensed):
            for line in condensed_lines:
                if cnt >= max_content:
                    break
                cnt += 1
                print(line, end='')

hexa_path = Path("output") / "big_pics" / "big_data_in_hexa.txt"
with open(hexa_path, "r") as f:
    cnt = 0
    lines = f.readlines()
    condensed_lines = []
    for line in lines:
        condensed_line = line.replace(" ", "")
        condensed_lines.append(condensed_line)
    condensed_hexa_path = Path("output") / "big_pics" / "big_data_in_hexa_condensed.txt"
    with open(condensed_hexa_path, "w") as f_condensed:
        with redirect_stdout(f_condensed):
            for line in condensed_lines:
                if cnt >= max_content:
                    break
                cnt += 1
                print(line, end='')

# then condense frames/ans/ans_hexa/*.txt
for a in range(frames.shape[0]):
    for b in range(frames.shape[1]):
        hexa_file = f"output/frames/ans/ans_hexa/ans_hexa_{a}_{b}.txt"
        with open(hexa_file, "r") as f:
            lines = f.readlines()
            condensed_lines = []
            for line in lines:
                condensed_line = line.replace(" ", "")
                condensed_lines.append(condensed_line)
            condensed_hexa_file = f"output/frames/ans/ans_hexa/ans_hexa_{a}_{b}_condensed.txt"
            with open(condensed_hexa_file, "w") as f_condensed:
                with redirect_stdout(f_condensed):
                    for line in condensed_lines:
                        print(line, end='')
# then condense frames/data_in/data_in_hexa/*.txt
for a in range(frames.shape[0]):
    for b in range(frames.shape[1]):
        hexa_file = f"output/frames/data_in/data_in_hexa/frame_hexa_{a}_{b}.txt"
        with open(hexa_file, "r") as f:
            lines = f.readlines()
            condensed_lines = []
            for line in lines:
                condensed_line = line.replace(" ", "")
                condensed_lines.append(condensed_line)
            condensed_hexa_file = f"output/frames/data_in/data_in_hexa/frame_hexa_{a}_{b}_condensed.txt"
            with open(condensed_hexa_file, "w") as f_condensed:
                with redirect_stdout(f_condensed):
                    for line in condensed_lines:
                        print(line, end='')